## Introduction and activity
Now that we have seen the combined power of analytical and numerical model to solve solute transport problems with solutes experiencing zero order, first order, sorption, and kinetic reactions, use these tools to model a process we have not yet explored.

Pair up and first program an analytical solution that we have not yet used and then build a corresponding 1D or 2D FloPy model to cross validate your models. Start by thinking about a scenario that you would like to understand better and then discuss how you would go about using these models to quantitatively describe transport in this scenario. 
Here are a few ideas:
- Build a first draft of your final project model,
- Replicate a model in one of the papers that we have read, 
- Or something else!

A good place to look for analytical solutions is the compilation by [van Genuchten and Alves, 1982](https://naldc.nal.usda.gov/download/CAT82780278/PDF), however many additional analytical solutions have been developed since then.

To begin here are a few useful blocks of code that we typically start with, feel free to look through past notebooks (for example the analytica solution used in the [First Order Reactions notebook](https://github.com/zahasky/Contaminant-Hydrogeology-Activities/blob/master/First%20Order%20Reactions.ipynb) for additional codes/functions to get started.

In [ ]:
# Import the flopy library
import flopy
# Import a few additional libraries
import pathlib
import sys
import os
# In addition to our typical libraries
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Path to MODFLOW executable, probably called 'mf2005'
exe_name_mf = 'C:\\Hydro\\MF2005.1_12\\bin\\mf2005'
# Print to make sure it is formated correctly
print(exe_name_mf)
# Path to MT3D executable, probably called 'mt3dms'
exe_name_mt = 'C:\\Hydro\\mt3dusgs1.1.0\\bin\\mt3d-usgs_1.1.0_64'
# Print to make sure it is formated correctly
print(exe_name_mt)

In [ ]:
# if this is not where you want to save stuff then make a new folder and set this to be the new working directory
# define path
path = pathlib.Path('C:\\Users\\zahas\\Dropbox\\Teaching\\Contaminant hydro 629\\Notebooks_unpublished')
# if folder doesn't exist then make it 
path.mkdir(parents=True, exist_ok=True)
# set working directory to this new folder
os.chdir(path)
current_directory = os.getcwd()
print(current_directory)
# directory to save data
directory_name = 'my_reaction_model'
# directory to save data
datadir = os.path.join('..', directory_name, 'mt3d_test', 'mt3dms')
workdir = os.path.join('.', directory_name)

Notice however that we don't yet name the folder where we will save data 'dirname'. This will be an input to our model function.


## Reactions Model Function
The first thing we do is setup the function. The input variables are:

### Function Input:
#### directory name
    direname = 

#### MT3D period length 
Time is in selected units, the model time length is the sum of this (for steady state flow it can be set to anything). Note that this is different from the MODFLOW flow model period input. Therefore the underlying flow conditions are constant and steady state across changes in MT3D stress periods. The format for multi-period input: ```[time_of_pulse_injection, time_of_rest_of_model]```

The function is set up to only allow for two input periods. For example:
 
    perlen_mt = [60, 60*58]
    
    
#### Porosity

    prsity = 
    
#### advection velocity

    v = 
    
#### dispersivity
Set the longitudinal dispersivity in selected units. What are the units again?

    al = 
  
### Reaction model input  
#### Model type
isothm is a flag indicating which type of sorption (or dual-domain mass transfer) is simulated: isothm = 0, no sorption is simulated; isothm = 1, linear isotherm (equilibrium-controlled); isothm = 2, Freundlich isotherm (equilibrium-controlled); isothm = 3, Langmuir isotherm (equilibrium-controlled); isothm = 4, first-order kinetic sorption (nonequilibrium); isothm = 5, dual-domain mass transfer (without sorption); isothm = 6, dual-domain mass transfer (with sorption).

    isothm = 

#### bulk density (rhob)
This can be a float or array of floats (nlay, nrow, ncol). rhob is the bulk density of the aquifer medium (unit, ML-3). rhob is used if isothm = 1, 2, 3, 4, or 6. If rhob is not user-specified and isothm is not 5 then rhob is set to 1.8e3. (default is None)


#### First sorption parameter (sp1)
Can be a float or array of floats (nlay, nrow, ncol). The use of sp1 depends on the type of sorption selected (the value of isothm). If sp1 is not specified and isothm > 0 then sp1 is set to 0. (default is None).

For linear sorption (isothm = 1) and nonequilibrium sorption (isothm = 4), sp1 is the distribution coefficient (Kd) (unit, L3M-1). 

For Freundlich sorption (isothm = 2), sp1 is the Freundlich equilibrium constant (Kf) (the unit depends on the Freundlich exponent a). 

For Langmuir sorption (isothm = 3), sp1 is the Langmuir equilibrium constant (Kl) (unit, L3M-1 ). 

For dual-domain mass transfer without sorption (isothm = 5), sp1 is not used, but still must be entered. For dual-domain mass transfer with sorption (isothm = 6), sp1 is also the distribution coefficient (Kd) (unit, L3M-1).

For example 
    
    isothm = 1
    sp1 = kd 

Where 
    
    kd = (retardation - 1.) * prsity / rhob

#### Second sorption parameter (sp2)
sp2 can be a float or array of floats (nlay, nrow, ncol). The use of sp2 depends on the type of sorption model selected. 

For linear sorption (isothm = 1), sp2 is read but not used. 

For Freundlich sorption (isothm = 2), sp2 is the Freundlich exponent N. 

For Langmuir sorption (isothm = 3), sp2 is the total concentration of the sorption sites available ( S ) (unit, MM-1).  If sp2 is not specified and isothm > 0 then sp2 is set to 0. (default is None).

For nonequilibrium sorption (isothm = 4), sp2 is the first-order mass transfer rate between the dissolved and sorbed phases (beta unit, T-1). 

For example 
    
    isothm = 2
    sp1 = kf
    sp2 = N


#### Additional Reaction Package input options (not used in this function 
ireact is an integer flag indicating which type of kinetic rate reaction is simulated: ireact = 0, no kinetic rate reaction is simulated

rc1 is the first-order reaction rate for the dissolved (liquid) phase for the first species (unit, T-1). rc1 is not used ireact = 0.  If a dual-domain system is simulated, the reaction rates for the liquid phase in the mobile and immobile domains are assumed to be equal. If rc1 is not specified and ireact > 0 then rc1 is set to 0. (default is None).

rc2 is the first-order reaction rate for the sorbed phase for the first species (unit, T-1). rc2 is not used ireact = 0. If a dual-domain system is simulated, the reaction rates for the sorbed phase in the mobile and immobile domains are assumed to be equal. Generally, if the reaction is radioactive decay, rc2 should be set equal to rc1, while for biodegradation, rc2 may be different from rc1. Note that rc2 is read but not used, if no sorption is included in the simulation. If rc2 is not specified and ireact > 0 then rc2 is set to 0. (default is None).